In [1]:
# https://keras.io/examples/generative/wgan_gp/
#

import os
os.environ["CUDA_VISIBLE_DEVICES"] = str(0)


import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import *

from tqdm import tqdm
import matplotlib.pyplot as plt
import glob
from sklearn.utils import shuffle

from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint

# example of loading the generator model and generating images
from numpy import asarray
from numpy.random import randn
from numpy.random import randint
from tensorflow.keras.models import load_model
from matplotlib import pyplot


import tensorflow as tf


from tensorflow.keras.datasets.fashion_mnist import load_data
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import BatchNormalization

from tensorflow.keras import initializers

from skimage import io
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import glob
from sklearn.utils import shuffle

In [2]:
foldername2class = {
	'0.0':  0,
	'0.05': 0,
	'0.1':  1,
	'0.15': 1,
	'0.2':  2,
	'0.25': 2,
	'0.3':  3,
	'0.35': 3,
	'0.4':  4,
	'0.45': 4,
	'0.5':  5,
	'0.55': 5,
	'0.6':  6,
	'0.65': 6,
	'0.7':  7,
	'0.75': 7,
	'0.8':  8,
	'0.85': 8,
	'0.9':  9,
	'0.95': 9,
	'1.0':  10,
}

In [3]:
def preprocess_images(images):
  images = (images - 127.5) / 127.5
  return images.astype('float32')

def generator_img(path_list: list):
    counter = 0
    max_counter = len(path_list)
    while True:
        single_path = path_list[counter]
        label_s = foldername2class[single_path.split('/')[-2]]
        image_s = preprocess_images(np.asarray(io.imread(single_path), dtype=np.float32))
        yield image_s, label_s
        # yield np.ones((336, 336, 3))
        counter += 1

        if counter == max_counter:
            counter = 0
            path_list = shuffle(path_list)

def train_gen():
    return generator_img(train_images_path)

In [4]:
IMG_SHAPE = (336, 336, 3)
BATCH_SIZE = 12
N_CLASSES = 11
# Size of the noise vector
noise_dim = 128

PATH_DATA = '../../expand_double_modes'
SAVE_RESULT = 'exp_result'

train_images_path = []

iterator = tqdm(glob.glob(PATH_DATA + "/*"))
for single_folder in iterator:
    img_folder = shuffle(glob.glob(single_folder + '/*'))
    for indx, single_img_path in enumerate(img_folder):
        train_images_path.append(single_img_path)
iterator.close()

train_images_path = shuffle(train_images_path)

100%|██████████| 21/21 [00:00<00:00, 145.17it/s]


In [5]:
dataset = (
    tf.data.Dataset.from_generator(
        train_gen, 
        output_signature=(
            tf.TensorSpec(shape=IMG_SHAPE, dtype=np.float32),
            tf.TensorSpec(shape=(), dtype=np.int32),
        )
    )
    .shuffle(BATCH_SIZE * 10).batch(BATCH_SIZE)
)


In [6]:
train_size = len(train_images_path)

print(f'train: {train_size}')

train: 37800


In [7]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples, n_classes=21):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    z_input = x_input.reshape(n_samples, latent_dim)
    # generate labels
    labels = randint(0, n_classes, n_samples)
    return [z_input, labels]

In [8]:
def init_weights():
    return initializers.RandomNormal(stddev=0.02)

def init_weights():
    return None

In [9]:
# define the standalone discriminator model
def define_discriminator(in_shape, n_classes=21):
    # label input
    in_label = Input(shape=(1,))
    # embedding for categorical input
    li = Embedding(n_classes, 96)(in_label)
    # scale up to image dimensions with linear activation
    n_nodes = in_shape[0] * in_shape[1] * in_shape[2]
    li = Dense(n_nodes)(li)
    # reshape to additional channel
    li = Reshape((in_shape[0], in_shape[1], in_shape[2]))(li)
    # image input
    in_image = Input(shape=in_shape)
    # concat label as a channel
    merge = Concatenate()([in_image, li])                         # 336
    # downsample
    fe = Conv2D(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init_weights())(merge) # 168
    #fe = BatchNormalization()(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    # downsample
    fe = Conv2D(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init_weights())(fe)    # 84  
    #fe = BatchNormalization()(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    # downsample
    fe = Conv2D(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init_weights())(fe)    # 42 
    #fe = BatchNormalization()(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    # downsample
    fe = Conv2D(256, (3,3), strides=(2,2), padding='same', kernel_initializer=init_weights())(fe)    # 21 
    #fe = BatchNormalization()(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    # downsample
    fe = Conv2D(256, (3,3), strides=(2,2), padding='same', kernel_initializer=init_weights())(fe)    # 10 
    #fe = BatchNormalization()(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    # flatten feature maps
    fe = layers.Flatten()(fe)
    fe = layers.Dropout(0.4)(fe)
    out_layer = layers.Dense(1)(fe)
    # define model
    model = keras.models.Model([in_image, in_label], out_layer)
    # compile model
    #opt = Adam(lr=0.0002, beta_1=0.5) # Adam(lr=0.0002, beta_1=0.5)
    #model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# define the standalone generator model
def define_generator(latent_dim, n_classes=21, h_low=21, w_low=21):
    # label input
    in_label = Input(shape=(1,))
    # embedding for categorical input
    li = Embedding(n_classes, 96)(in_label)
    # linear multiplication
    n_nodes = h_low * w_low
    li = Dense(n_nodes)(li)
    # reshape to additional channel
    li = Reshape((h_low, w_low, 1))(li)
    # image generator input
    in_lat = Input(shape=(latent_dim,))
    # foundation for h_low x w_low image
    n_nodes = 64 * h_low * w_low
    gen = Dense(n_nodes)(in_lat)
    gen = LeakyReLU(alpha=0.2)(gen)
    gen = Reshape((h_low, w_low, 64))(gen)
    # merge image gen and label input
    merge = Concatenate()([gen, li])
    # upsample to 42
    gen = Conv2DTranspose(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init_weights())(merge)  
    #gen = BatchNormalization(trainable=False)(gen)
    gen = LeakyReLU(alpha=0.2)(gen)
    # upsample to 84
    gen = Conv2DTranspose(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init_weights())(gen)
    #gen = BatchNormalization(trainable=False)(gen)
    gen = LeakyReLU(alpha=0.2)(gen)
    # upsample to 168
    gen = Conv2DTranspose(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init_weights())(gen)
    #gen = BatchNormalization(trainable=False)(gen)
    gen = LeakyReLU(alpha=0.2)(gen)
    # upsample to 336
    gen = Conv2DTranspose(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init_weights())(gen)
    #gen = BatchNormalization(trainable=False)(gen)
    gen = LeakyReLU(alpha=0.2)(gen)
    # output
    out_layer = Conv2D(IMG_SHAPE[-1], (3,3), activation='tanh', padding='same', kernel_initializer=init_weights())(gen)
    # define model
    model = keras.models.Model([in_lat, in_label], out_layer)
    return model

In [10]:
def conv_block(
    x,
    filters,
    activation,
    kernel_size=(3, 3),
    strides=(1, 1),
    padding="same",
    use_bias=True,
    use_bn=False,
    use_dropout=False,
    drop_value=0.5,
):
    x = layers.Conv2D(
        filters, kernel_size, strides=strides, padding=padding, use_bias=use_bias
    )(x)
    if use_bn:
        x = layers.BatchNormalization()(x)
    x = activation(x)
    if use_dropout:
        x = layers.Dropout(drop_value)(x)
    return x


def get_discriminator_model():
    img_input = layers.Input(shape=IMG_SHAPE)
    # Zero pad the input to make the input images size to (32, 32, 1).
    x = layers.ZeroPadding2D((2, 2))(img_input)
    x = conv_block(
        x,
        64,
        kernel_size=(5, 5),
        strides=(2, 2),
        use_bn=False,
        use_bias=True,
        activation=layers.LeakyReLU(0.2),
        use_dropout=False,
        drop_value=0.3,
    )
    x = conv_block(
        x,
        128,
        kernel_size=(5, 5),
        strides=(2, 2),
        use_bn=False,
        activation=layers.LeakyReLU(0.2),
        use_bias=True,
        use_dropout=True,
        drop_value=0.3,
    )
    x = conv_block(
        x,
        256,
        kernel_size=(5, 5),
        strides=(2, 2),
        use_bn=False,
        activation=layers.LeakyReLU(0.2),
        use_bias=True,
        use_dropout=True,
        drop_value=0.3,
    )
    x = conv_block(
        x,
        512,
        kernel_size=(5, 5),
        strides=(2, 2),
        use_bn=False,
        activation=layers.LeakyReLU(0.2),
        use_bias=True,
        use_dropout=False,
        drop_value=0.3,
    )

    x = layers.Flatten()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(1)(x)

    d_model = keras.models.Model(img_input, x, name="discriminator")
    return d_model


d_model = define_discriminator(IMG_SHAPE, n_classes=N_CLASSES)
d_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1, 96)        1056        input_1[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 1, 338688)    32852736    embedding[0][0]                  
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 336, 336, 3) 0                                            
______________________________________________________________________________________________

In [11]:
def upsample_block(
    x,
    filters,
    activation,
    kernel_size=(3, 3),
    strides=(1, 1),
    up_size=(2, 2),
    padding="same",
    use_bn=False,
    use_bias=True,
    use_dropout=False,
    drop_value=0.3,
):
    x = layers.UpSampling2D(up_size)(x)
    x = layers.Conv2D(
        filters, kernel_size, strides=strides, padding=padding, use_bias=use_bias
    )(x)

    if use_bn:
        x = layers.BatchNormalization()(x)

    if activation:
        x = activation(x)
    if use_dropout:
        x = layers.Dropout(drop_value)(x)
    return x


def get_generator_model():
    noise = layers.Input(shape=(noise_dim,))
    x = layers.Dense(4 * 4 * 256, use_bias=False)(noise)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Reshape((4, 4, 256))(x)
    x = upsample_block(
        x,
        128,
        layers.LeakyReLU(0.2),
        strides=(1, 1),
        use_bias=False,
        use_bn=True,
        padding="same",
        use_dropout=False,
    )
    x = upsample_block(
        x,
        64,
        layers.LeakyReLU(0.2),
        strides=(1, 1),
        use_bias=False,
        use_bn=True,
        padding="same",
        use_dropout=False,
    )
    x = upsample_block(
        x, 1, layers.Activation("tanh"), strides=(1, 1), use_bias=False, use_bn=True
    )
    # At this point, we have an output which has the same shape as the input, (32, 32, 1).
    # We will use a Cropping2D layer to make it (28, 28, 1).
    x = layers.Cropping2D((2, 2))(x)

    g_model = keras.models.Model(noise, x, name="generator")
    return g_model


g_model = define_generator(noise_dim, n_classes=N_CLASSES)
g_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 28224)        3640896     input_4[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 96)        1056        input_3[0][0]                    
____________________________________________________________________________________________

In [12]:
class WGAN(keras.Model):
    def __init__(
        self,
        discriminator,
        generator,
        latent_dim,
        discriminator_extra_steps=3,
        gp_weight=10.0,
    ):
        super(WGAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.d_steps = discriminator_extra_steps
        self.gp_weight = gp_weight

    def compile(self, d_optimizer, g_optimizer, d_loss_fn, g_loss_fn):
        super(WGAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.d_loss_fn = d_loss_fn
        self.g_loss_fn = g_loss_fn

    def gradient_penalty(self, batch_size, real_images, fake_images, real_labels):
        """ Calculates the gradient penalty.

        This loss is calculated on an interpolated image
        and added to the discriminator loss.
        """
        # Get the interpolated image
        alpha = tf.random.normal([batch_size, 1, 1, 1], 0.0, 1.0)
        diff = fake_images - real_images
        interpolated = real_images + alpha * diff

        with tf.GradientTape() as gp_tape:
            gp_tape.watch(interpolated)
            # 1. Get the discriminator output for this interpolated image.
            pred = self.discriminator([interpolated, real_labels], training=True)

        # 2. Calculate the gradients w.r.t to this interpolated image.
        grads = gp_tape.gradient(pred, [interpolated, real_labels])[0]
        # 3. Calculate the norm of the gradients.
        norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=[1, 2, 3]))
        gp = tf.reduce_mean((norm - 1.0) ** 2)
        return gp

    def train_step(self, data):
        real_images, real_labels = data

        # Get the batch size
        batch_size = tf.shape(real_images)[0]

        # For each batch, we are going to perform the
        # following steps as laid out in the original paper:
        # 1. Train the generator and get the generator loss
        # 2. Train the discriminator and get the discriminator loss
        # 3. Calculate the gradient penalty
        # 4. Multiply this gradient penalty with a constant weight factor
        # 5. Add the gradient penalty to the discriminator loss
        # 6. Return the generator and discriminator losses as a loss dictionary

        # Train the discriminator first. The original paper recommends training
        # the discriminator for `x` more steps (typically 5) as compared to
        # one step of the generator. Here we will train it for 3 extra steps
        # as compared to 5 to reduce the training time.
        for i in range(self.d_steps):
            # Get the latent vector
            random_latent_vectors = tf.random.normal(
                shape=(batch_size, self.latent_dim)
            )
            with tf.GradientTape() as tape:
                # Generate fake images from the latent vector
                fake_images = self.generator([random_latent_vectors, real_labels], training=True)
                # Get the logits for the fake images
                fake_logits = self.discriminator([fake_images, real_labels], training=True)
                # Get the logits for the real images
                real_logits = self.discriminator([real_images, real_labels], training=True)

                # Calculate the discriminator loss using the fake and real image logits
                d_cost = self.d_loss_fn(real_img=real_logits, fake_img=fake_logits)
                # Calculate the gradient penalty
                gp = self.gradient_penalty(batch_size, real_images, fake_images, real_labels)
                # Add the gradient penalty to the original discriminator loss
                d_loss = d_cost + gp * self.gp_weight

            # Get the gradients w.r.t the discriminator loss
            d_gradient = tape.gradient(d_loss, self.discriminator.trainable_variables)
            # Update the weights of the discriminator using the discriminator optimizer
            self.d_optimizer.apply_gradients(
                zip(d_gradient, self.discriminator.trainable_variables)
            )

        # Train the generator
        # Get the latent vector
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        random_labels = np.random.randint(0, N_CLASSES, batch_size)
        with tf.GradientTape() as tape:
            # Generate fake images using the generator
            generated_images = self.generator([random_latent_vectors, random_labels], training=True)
            # Get the discriminator logits for fake images
            gen_img_logits = self.discriminator([generated_images, random_labels], training=True)
            # Calculate the generator loss
            g_loss = self.g_loss_fn(gen_img_logits)

        # Get the gradients w.r.t the generator loss
        gen_gradient = tape.gradient(g_loss, self.generator.trainable_variables)
        # Update the weights of the generator using the generator optimizer
        self.g_optimizer.apply_gradients(
            zip(gen_gradient, self.generator.trainable_variables)
        )
        return {"d_loss": d_loss, "g_loss": g_loss}

In [13]:
class GANMonitor():
    def __init__(self, model, num_img=100, latent_dim=128):
        self.num_img = num_img
        self.latent_dim = latent_dim
        self.model = model

    def on_epoch_end(self, epoch, logs=None, save_path=''):
        n = int(np.sqrt(self.num_img))
        random_latent_vectors = np.random.normal(size=(self.num_img, self.latent_dim))
        random_labels = np.asarray([x for _ in range(10) for x in range(10)])
        generated_images = self.model.predict([random_latent_vectors, random_labels])
        # scale from [-1,1] to [0,1]
        generated_images = (generated_images + 1) / 2.0
        self._generate_plot(generated_images, n, os.path.join(save_path, f'{epoch}'))
    
    def _generate_plot(self, examples, n, prefix):
        # plot images
        fig = plt.figure(figsize=(12,12))
        for i in range(n * n):
            # define subplot
            plt.subplot(n, n, 1 + i)
            # turn off axis
            plt.axis('off')
            # plot raw pixel data
            plt.imshow(examples[i])
        #pyplot.show()
        fig.savefig(f'{prefix}_image.png')
        plt.close('all')

In [14]:
# Instantiate the optimizer for both networks
# (learning_rate=0.0002, beta_1=0.5 are recommended)
generator_optimizer = keras.optimizers.Adam(
    learning_rate=0.0002, beta_1=0.5, beta_2=0.9
)
discriminator_optimizer = keras.optimizers.Adam(
    learning_rate=0.0002, beta_1=0.5, beta_2=0.9
)

# Define the loss functions for the discriminator,
# which should be (fake_loss - real_loss).
# We will add the gradient penalty later to this loss function.
def discriminator_loss(real_img, fake_img):
    real_loss = tf.reduce_mean(real_img)
    fake_loss = tf.reduce_mean(fake_img)
    return fake_loss - real_loss


# Define the loss functions for the generator.
def generator_loss(fake_img):
    return -tf.reduce_mean(fake_img)


# Set the number of epochs for trainining.
epochs = 20

# Instantiate the customer `GANMonitor` Keras callback.
cbk = GANMonitor(g_model, num_img=100, latent_dim=noise_dim)

# Instantiate the WGAN model.
wgan = WGAN(
    discriminator=d_model,
    generator=g_model,
    latent_dim=noise_dim,
    discriminator_extra_steps=5, # was 3
)

# Compile the WGAN model.
wgan.compile(
    d_optimizer=discriminator_optimizer,
    g_optimizer=generator_optimizer,
    g_loss_fn=generator_loss,
    d_loss_fn=discriminator_loss,
)

# Start training the model.
#wgan.fit(train_images, batch_size=BATCH_SIZE, epochs=epochs, callbacks=[cbk])

In [ ]:
for ep in range(epochs):
    iteration = train_size // BATCH_SIZE
    save_path = os.path.join(SAVE_RESULT, f'ep_{ep}')
    os.makedirs(save_path, exist_ok=True)
    for i in range(iteration):
        real_images, real_labels = list(dataset.take(1))[0]
        data_losses = wgan.train_step([real_images, real_labels])
        print('>%d, %d/%d, d=%.3f, g=%.3f' %
            (ep+1, i+1, iteration, data_losses['d_loss'], data_losses['g_loss']))
        
        if i % 50 == 0:
            cbk.on_epoch_end(f'i_{i}_ep_{ep}', save_path=save_path)

>1, 1/3150, d=4.842, g=0.035
>1, 2/3150, d=-96.504, g=0.433
>1, 3/3150, d=-1293.925, g=3.076
>1, 4/3150, d=-4708.042, g=0.224
>1, 5/3150, d=-5701.115, g=-7.335
>1, 6/3150, d=-8033.316, g=-3.931
>1, 7/3150, d=-8343.949, g=-17.904
>1, 8/3150, d=-7424.716, g=-24.874
>1, 9/3150, d=-9882.123, g=-36.429
>1, 10/3150, d=-6516.838, g=-64.805
>1, 11/3150, d=-7625.463, g=-38.666


>1, 12/3150, d=-6870.891, g=-116.948
>1, 13/3150, d=-5684.982, g=-131.670
>1, 14/3150, d=-6685.491, g=-472.813
>1, 15/3150, d=-6852.980, g=-898.525
>1, 16/3150, d=-4800.245, g=-1106.697
>1, 17/3150, d=-4095.655, g=-1216.958
>1, 18/3150, d=-3104.929, g=-1293.664
>1, 19/3150, d=-3004.169, g=-1658.659
>1, 20/3150, d=-2213.023, g=-1511.317
>1, 21/3150, d=-1706.782, g=-1180.263
>1, 22/3150, d=-1736.997, g=-973.093
>1, 23/3150, d=-1195.949, g=-113.790


>1, 24/3150, d=-1199.308, g=-326.105
>1, 25/3150, d=-977.885, g=168.683
>1, 26/3150, d=-981.878, g=121.286
>1, 27/3150, d=-880.734, g=188.610
>1, 28/3150, d=-617.141, g=57.807
>1, 29/3150, d=-1182.825, g=963.141
>1, 30/3150, d=-768.596, g=804.426
>1, 31/3150, d=-943.405, g=1234.717
>1, 32/3150, d=-1139.490, g=351.608
>1, 33/3150, d=-1047.321, g=1750.049
>1, 34/3150, d=-1378.061, g=1979.426
>1, 35/3150, d=-1371.122, g=2687.538


>1, 36/3150, d=-1266.022, g=3488.501
>1, 37/3150, d=-1070.711, g=1973.356
>1, 38/3150, d=-1460.237, g=2393.080
>1, 39/3150, d=-1109.819, g=1940.416
>1, 40/3150, d=-1289.469, g=3333.547
>1, 41/3150, d=-1274.152, g=3703.711
>1, 42/3150, d=-1418.175, g=2713.232
>1, 43/3150, d=-1147.599, g=2211.461
>1, 44/3150, d=-1428.155, g=4504.516
>1, 45/3150, d=-1411.252, g=5012.891
>1, 46/3150, d=-1236.240, g=3992.415


>1, 47/3150, d=-1463.938, g=5036.921
>1, 48/3150, d=-1423.657, g=4795.839
>1, 49/3150, d=-1776.992, g=6546.834
>1, 50/3150, d=-1261.964, g=4861.276
>1, 51/3150, d=-1335.640, g=4865.853
>1, 52/3150, d=-1483.104, g=4195.425
>1, 53/3150, d=-1591.538, g=6121.755
>1, 54/3150, d=-950.548, g=4958.877
>1, 55/3150, d=-1197.250, g=6116.158
>1, 56/3150, d=-1042.296, g=5993.996
>1, 57/3150, d=-1235.790, g=6201.815
>1, 58/3150, d=-1407.666, g=5419.846


...

>1, 2216/3150, d=-67.369, g=-319.692
>1, 2217/3150, d=-302.572, g=-210.001
>1, 2218/3150, d=151.640, g=-472.665
>1, 2219/3150, d=-106.879, g=-503.312
>1, 2220/3150, d=-116.541, g=-393.019
>1, 2221/3150, d=-178.119, g=-424.884
>1, 2222/3150, d=24.666, g=-425.929
>1, 2223/3150, d=-20.074, g=-211.785
>1, 2224/3150, d=-189.269, g=-452.449
>1, 2225/3150, d=-210.769, g=33.647
>1, 2226/3150, d=-248.925, g=39.510
>1, 2227/3150, d=-269.370, g=-464.759


>1, 2228/3150, d=-298.301, g=-856.236
>1, 2229/3150, d=-80.948, g=-101.065
>1, 2230/3150, d=-22.107, g=-752.949
>1, 2231/3150, d=-80.794, g=-206.602
>1, 2232/3150, d=-147.582, g=-100.314
>1, 2233/3150, d=-67.747, g=-266.144
>1, 2234/3150, d=-62.712, g=-1.377
>1, 2235/3150, d=-79.942, g=-286.134
>1, 2236/3150, d=-180.579, g=-255.553
>1, 2237/3150, d=-163.426, g=-582.868
>1, 2238/3150, d=-160.863, g=-261.059
>1, 2239/3150, d=-165.597, g=-456.981


>1, 2240/3150, d=-429.180, g=-46.779
>1, 2241/3150, d=-447.886, g=-204.603
>1, 2242/3150, d=-54.465, g=-439.221
>1, 2243/3150, d=-363.822, g=-670.347
>1, 2244/3150, d=-288.496, g=222.297
>1, 2245/3150, d=-496.381, g=-386.512
>1, 2246/3150, d=164.427, g=-709.231
>1, 2247/3150, d=-495.426, g=-1059.492
>1, 2248/3150, d=119.245, g=-901.062
>1, 2249/3150, d=-704.662, g=134.362
>1, 2250/3150, d=-106.298, g=-130.123


>1, 2251/3150, d=-837.542, g=-721.290
>1, 2252/3150, d=-1.496, g=-1780.224
>1, 2253/3150, d=417.854, g=-899.121
>1, 2254/3150, d=-346.208, g=-1469.052
>1, 2255/3150, d=-372.162, g=-1876.490
>1, 2256/3150, d=-470.148, g=-979.150
>1, 2257/3150, d=-61.860, g=-735.194
>1, 2258/3150, d=-230.455, g=-1077.088
>1, 2259/3150, d=-275.832, g=-319.408
>1, 2260/3150, d=-174.282, g=-332.116
>1, 2261/3150, d=101.834, g=175.804
>1, 2262/3150, d=68.029, g=480.814


>1, 2263/3150, d=-51.670, g=-195.556
>1, 2264/3150, d=37.315, g=95.356


In [ ]:
wgan.save_weights('wgan_generator.h5')